## 一、构建词典
### 1.1 整理数据
把05-23年期间的业绩说明会问答文本汇总到一个txt文件内。为了保证问答上下文一致， 问答要放在相邻处。可能需要安装


```
pip3 install openpyxl
pip3 install pandas
```

In [109]:
import pandas as pd

df = pd.read_excel('业绩说明会问答05-23.xlsx')
df['text'] = df['提问内容']+df['回答内容']
df.dropna(subset=['text'], inplace=True)
df.head()

,股票代码,会计年度,问题序号,提问内容,提问时间,回答人,回答时间,回答内容,text
0,A000429,2005,1,"请问肖来久：公司将考虑通过资产证券化进行融资吗?是怎么回事,能否给我解释解释?谢谢",2006-03-31 15:00~17:00,肖来久,2006-03-31 15:00~17:00,资产证券化将会在国内的资本市场里扮演越来越重要的角色，公司的相关业务部门正对这种新兴的融资办...,"请问肖来久：公司将考虑通过资产证券化进行融资吗?是怎么回事,能否给我解释解释?谢谢资产证券化..."
1,A000429,2005,2,请问曹晓峰：您好！我在1994年购买了佛开法人股和1997年购买了广东省高速公路股份有限公司...,2006-03-31 15:00~17:00,曹晓峰,2006-03-31 15:00~17:00,请打电话至83731388-233、221咨询。谢谢,请问曹晓峰：您好！我在1994年购买了佛开法人股和1997年购买了广东省高速公路股份有限公司...
2,A000429,2005,3,请问曹晓峰：过去三年的利润增长率分别为多少?,2006-03-31 15:00~17:00,肖来久,2006-03-31 15:00~17:00,公司2003-2005年净利润同比增长为12.77%，31.27%，41.58%。,请问曹晓峰：过去三年的利润增长率分别为多少?公司2003-2005年净利润同比增长为12.7...
3,A000429,2005,4,国资委说过要考核市值，你们做了吗？,2006-03-31 15:00~17:00,肖来久,2006-03-31 15:00~17:00,目前公司还没有收到国资委的有关通知。如果国资委要考核市值，也应该是对控股股东广东省交通集团有...,国资委说过要考核市值，你们做了吗？目前公司还没有收到国资委的有关通知。如果国资委要考核市值，...
4,A000429,2005,5,最近两年汽车保有量开始显著上升，但为什么全国高速公路车流量仍未出现明显增速？您预计因为汽车保...,2006-03-31 15:00~17:00,霍燕滨,2006-03-31 15:00~17:00,高速公路车流量的增长与当地经济的发达程度，高速公路所出的位置等因素相关。从我司参、控股的各条...,最近两年汽车保有量开始显著上升，但为什么全国高速公路车流量仍未出现明显增速？您预计因为汽车保...


In [ ]:
with open('业绩说明会05-23.txt', 'a+', encoding='utf-8') as f:
    
    #为了保证问答上下文一致， 问答要放在相邻处
    text = ''.join(df['text'].to_list())
    f.write(text)

<br>

### 1.2 训练word2vec
一般都是使用gensim库，对「业绩说明会05-23.txt」数据集进行训练，我已经封装到cntext库内。

```
pip3 install cntext --upgrade
```

In [ ]:
import cntext as ct
import os


#Init W2VModels. Support English and Chinese
model = ct.W2VModels(cwd=os.getcwd(), 
                     lang='chinese')  #corpus data w2v_corpus.txt


#训练结束后，「业绩说明会05-23.100.6.bin」会出现在「output/word2vec」文件夹内 
model.train(input_txt_file='业绩说明会05-23.txt', 
            model_name='业绩说明会05-23.100.6.bin')

<br>

需要注意， 文件夹内会同时含有 
- 业绩说明会05-23.100.6.bin
- 业绩说明会05-23.100.6.bin.vectors.npy 

两个文件都不要删除， 这些是预训练词向量文件。

<br>

### 1.3 扩展词典
根据前瞻性研究需要，整理了一些种子词

```python
seedwords = ['计划', '预计', '未来', '目标', '可能', '如果', '机遇', '预期', '挑战', '预测', '今后', '目的', '契机', '前景', '希望', '展望', '相信', '愿景', '期待', '明年', '期望']
```

1. 导入word2vec预训练语言模型文件**业绩说明会05-23.100.6.bin**
2. 寻找与种子词语义最相似的n个词。 
3. 经过人工检查，剔除n个词中与**前瞻性**无关的词语，最终得到**前瞻性词典**(论文中是174个词)。


>但是，经过大邓测试发现业绩说明会训练得到的业绩说明会word2vec(05-23.100.6.bin)模型表现很差。之前大邓用01-21年管理层讨论与分析训练过一个word2vec(mda01-21.200.6.bin)，在这次扩展词任务中，mda01-21.200.6.bin表现要远好于05-23.100.6.bin。

In [103]:
from gensim.models import KeyedVectors

def load_w2v(w2v_path):
    """
    Load word2vec model

    Args:
        w2v_path (str): path of word2vec model

    Returns:
        model: word2vec model
    """
    print('Loading word2vec model...')
    model = KeyedVectors.load(w2v_path)
    return model


wv = load_w2v('Embeddings/业绩说明会05-23.100.6.bin')
wv2 = load_w2v('Embeddings/mda01-21.200.6.bin')

Loading word2vec model...
Loading word2vec model...


In [104]:
#词汇量
print(len(wv.index_to_key))
print(len(wv2.index_to_key))

198776
789539


In [8]:
#查询某词的词向量
wv.get_vector('创新')

#查询多个词的词向量
#wv.get_mean_vector(['创新', '研发'])

array([ 0.43675017,  0.74739504,  3.3765798 , -0.29287583,  0.40125442,
        0.9364979 ,  0.62465197,  0.06480039,  0.12256158, -2.0735328 ,
       -0.256066  , -1.7680115 , -0.8514873 , -0.756108  ,  1.3441261 ,
       -0.18098126,  2.7290103 , -4.6596766 ,  0.4046495 , -4.0644083 ,
        0.6022293 ,  1.3569978 ,  1.0036035 ,  0.06123297, -2.0733726 ,
        2.2704456 , -1.2935334 , -0.2855776 ,  1.588003  ,  1.5027634 ,
        2.0897112 , -0.8861778 ,  0.4014722 , -0.41474393, -1.5390201 ,
        0.23899865, -0.9823706 , -2.986944  , -2.6887195 , -2.2386284 ,
        0.04810223,  1.3241886 , -0.71262985, -0.8015585 ,  1.5249555 ,
       -3.611584  , -1.4187033 , -1.6014036 ,  0.816903  ,  3.1821172 ,
       -1.7302881 , -0.8280679 , -1.2833163 ,  0.65565586, -0.8857021 ,
        2.098562  ,  1.4773984 ,  1.0931807 , -0.02242889,  1.1279039 ,
       -2.2318523 ,  0.24540211,  0.17126203,  2.5631666 , -1.7135285 ,
        0.60896975, -0.2654438 ,  0.5718087 , -1.4996717 ,  1.01

In [39]:
def expand_dictionary(wv, seedwords, topn=100):
    """
    According to the seed word file, select the top n words with the most similar semantics and save them in the directory save_dir.
    
    Args:
        wv (Word2VecKeyedVectors): the word embedding model
        seedwords (list): 种子词
        topn (int, optional): Set the number of most similar words to retrieve to topn. Defaults to 100.
        save_dir (str, optional): the directory to save the candidate words. Defaults to 'Word2Vec'.
    
    Returns:
    """
    simidx_scores = []

    similars_candidate_idxs = [] #the candidate words of seedwords
    dictionary = wv.key_to_index
    seedidxs = [] #transform word to index
    for seed in seedwords:
        if seed in dictionary:
            seedidx = dictionary[seed]
            seedidxs.append(seedidx)
    for seedidx in seedidxs:
        # sims_words such as [('by', 0.99984), ('or', 0.99982), ('an', 0.99981), ('up', 0.99980)]
        sims_words = wv.similar_by_word(seedidx, topn=topn)
        #Convert words to index and store them
        similars_candidate_idxs.extend([dictionary[sim[0]] for sim in sims_words])
    similars_candidate_idxs = set(similars_candidate_idxs)
    
    for idx in similars_candidate_idxs:
        score = wv.n_similarity([idx], seedidxs)
        simidx_scores.append((idx, score))
    simidxs = [w[0] for w in sorted(simidx_scores, key=lambda k:k[1], reverse=True)]

    simwords = [str(wv.index_to_key[idx]) for idx in simidxs][:topn]

    resultwords = []
    resultwords.extend(seedwords)
    resultwords.extend(simwords)
    
    return resultwords


#为了节省板面，这里设置为50
#论文中经过筛选留下174个词，实际上topn应该远大于174，
expand_dictionary(wv=wv, 
                  #前瞻性种子词
                  seedwords= ['计划', '预计', '未来', '目标', '可能', '如果', '机遇', '预期', '挑战', '预测', '今后', '目的', '契机', '前景', '希望', '展望', '相信', '愿景', '期待', '明年', '期望'],
                  topn=50)

['计划',
 '预计',
 '未来',
 '目标',
 '可能',
 '如果',
 '机遇',
 '预期',
 '挑战',
 '预测',
 '今后',
 '目的',
 '契机',
 '前景',
 '希望',
 '展望',
 '相信',
 '愿景',
 '期待',
 '明年',
 '期望',
 '几年',
 '积极影响',
 '有何',
 '谢谢您提问',
 '今后',
 '这块',
 '近几年',
 '近两年',
 '请问李',
 '裁员',
 '亮点',
 '准备采取',
 '将会',
 '接下来',
 '有何规划',
 '前景',
 '管理层是否',
 '未来几年',
 '有没有新',
 '发展状况',
 '一块',
 '当前',
 '很大',
 '这块业务',
 'LNG船',
 '具体措施您好',
 '当下',
 '是否能够',
 '明后',
 '一个台阶',
 '是否符合',
 '巨大',
 '预判',
 '对此',
 '未来三年',
 '资本开支',
 '不少',
 '未来是否',
 '这方面',
 '看法',
 '今年以来',
 '疫情结束',
 '想知道',
 '取得不错',
 '谈谈',
 '一步',
 '今年是否',
 '发展前景',
 '东宝',
 '现状']

In [40]:
expand_dictionary(wv=wv2, 
                  #前瞻性种子词
                  seedwords= ['计划', '预计', '未来', '目标', '可能', '如果', '机遇', '预期', '挑战', '预测', '今后', '目的', '契机', '前景', '希望', '展望', '相信', '愿景', '期待', '明年', '期望'],
                  topn=50)

['计划',
 '预计',
 '未来',
 '目标',
 '可能',
 '如果',
 '机遇',
 '预期',
 '挑战',
 '预测',
 '今后',
 '目的',
 '契机',
 '前景',
 '希望',
 '展望',
 '相信',
 '愿景',
 '期待',
 '明年',
 '期望',
 '相信',
 '将会',
 '未来',
 '希望',
 '预见',
 '预期',
 '可能',
 '必将',
 '应该',
 '未来几年',
 '今后',
 '有望',
 '目标',
 '这一',
 '当前',
 '当下',
 '无疑',
 '期望',
 '接下来',
 '意味着',
 '背景',
 '期待',
 '近期',
 '下一阶段',
 '机会',
 '看到',
 '预示',
 '能够',
 '短期内',
 '未来一段时间',
 '将来',
 '展望未来',
 '必须',
 '真正',
 '眼光',
 '必然',
 '还会',
 '预计',
 '未来十年',
 '机遇',
 '可能性',
 '后续',
 '潜在',
 '决心',
 '信心',
 '仍然',
 '非常',
 '这为',
 '未来五年',
 '短时间']

In [ ]:
在这里大邓假装经过很多检查，剔除不相关词语，最终跟论文一样，都得到了174个前瞻性词语。  需要说明，大邓已经将174个词

In [73]:
#cntext已内置了论文中的174个前瞻性词集
import cntext as ct

fls_words = ct.load_pkl_dict('Chinese_FLS.pkl')['Chinese_FLS']
fls_words

['计划',
 '预计',
 '未来',
 '目标',
 '可能',
 '如果',
 '机遇',
 '预期',
 '挑战',
 '预测',
 '今后',
 '目的',
 '契机',
 '前景',
 '希望',
 '展望',
 '相信',
 '愿景',
 '期待',
 '明年',
 '期望',
 '战略规划',
 '规划',
 '开发计划',
 '年度计划',
 '时间表',
 '战略部署',
 '分步',
 '意向',
 '计划指',
 '标',
 '打算',
 '收购计划',
 '估计',
 '预估',
 '陆续',
 '如期',
 '年内',
 '有望',
 '届时',
 '后期',
 '将来',
 '今年',
 '后续',
 '中长期',
 '日后',
 '今明两年',
 '长远',
 '战略目标',
 '总体目标',
 '方针',
 '既定目标',
 '奋斗目标',
 '战略方针',
 '战略',
 '思路',
 '力争',
 '必然',
 '深远影响',
 '威胁',
 '可能性',
 '转机',
 '在短期内',
 '障碍',
 '不确定性',
 '一旦',
 '假如',
 '只要',
 '如若',
 '也许',
 '新机遇',
 '良机',
 '机会',
 '商机',
 '机遇期',
 '新动向',
 '趋势',
 '动向',
 '预定',
 '期望值',
 '承诺',
 '长远规划',
 '增长速度',
 '困难',
 '压力',
 '诸多困难',
 '考验',
 '严峻形势',
 '不利因素',
 '风险',
 '需求',
 '难题',
 '变革',
 '迎',
 '接',
 '对策',
 '测算',
 '估算',
 '预报',
 '判断',
 '预告',
 '推算',
 '评判',
 '推断',
 '预知',
 '接下来',
 '以后',
 '后面',
 '有效途径',
 '手段',
 '路径',
 '重要途径',
 '先决条件',
 '必备条件',
 '突破口',
 '切入点',
 '高潮',
 '新起点',
 '创造条件',
 '有利条件',
 '大好时机',
 '力量',
 '抓手',
 '发展前景',
 '市场前景',
 '成长性',
 '发展势头',
 '发展前途',
 '前途',
 '发展潜力',
 '市场潜力',
 '空间',
 '

In [68]:
len(fls_words)

174

## 二、计算前瞻性

1. 汇总记录； 将同一年同一家上市公司的所有问答合并为一条记录，存储于df2中。
2. 设计前瞻性计算函数 **compute_fls**
3. 对df2['text']使用前瞻性计算函数**compute_fls**，计算结果保存到字段**Forward**


### 2.1 汇总记录

将同一年同一家上市公司的所有问答合并为一条记录，存储于新df的text中。

In [110]:
df2 = df.groupby(['股票代码', '会计年度'])['text'].apply(''.join).reset_index()
df2.dropna(subset=['text'], inplace=True)

df2

,股票代码,会计年度,text
0,A000002,2008,请问王文金：请问一下，公司有近200亿的现金，而公司短期负债加长期负债和公司债只有194亿，...
1,A000002,2009,绿色住宅是未来的方向，请问公司是如何肯定的来确定？据媒体报道今年冬季的气温并没有直接显示气候...
2,A000002,2010,"请问：王石董事长若离开万科,那万科会受到哪些影响呢?您好。王石主席并没有离开万科，在哈佛求学..."
3,A000002,2011,请问：在年报中提到２０１２年的结算收入的毛利率会有所下降，净利润会影响几个百分点．与２０１１...
4,A000002,2012,万科的房子都是委托给谁来盖？公司没有下属的建筑公司，公司开发的物业都是外包给建筑公司承建，监...
...,...,...,...
19751,A839725,2022,请问公司自产钻石是否已经出货。对外销售还是自用。主要用于什么领域。谢谢！尊敬的投资者您好：公...
19752,A870866,2022,券商在持续督导期主要做哪些工作？您好，在持续督导期间，保荐机构通过日常沟通、定期回访、现场检...
19753,A872190,2022,今年有新开设品牌形象店的计划吗？请详细介绍一下。尊敬的投资者，您好，公司募投项目中，会开设1...
19754,A873001,2022,公司2022年业绩下降原因，尊敬的投资者，您好！公司2022年度营业收入2.66亿元，同比下...


<br> 

### 2.2  设计前瞻性计算函数compute_fls


In [115]:
import cntext as ct
import jieba


fls_words = ct.load_pkl_dict('Chinese_FLS.pkl')['Chinese_FLS']
stopwords = ct.load_pkl_dict('STOPWORDS.pkl')['STOPWORDS']['chinese']

def compute_fls(text):
    num = 0
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords]
    for word in words:
        if word in fls_words:
            num+=1
    #+1是为了防止分母为0的情况
    return num/(len(words)+1)




<br>


### 2.3 批量计算df2['text']

对df2['text']批量使用前瞻性计算函数**compute_fls**，计算结果保存到字段**Forward**

In [116]:
df2['Forward'] = df2['text'].apply(compute_fls)

df2.head()

,股票代码,会计年度,text,Forward
0,A000002,2008,请问王文金：请问一下，公司有近200亿的现金，而公司短期负债加长期负债和公司债只有194亿，...,0.035213
1,A000002,2009,绿色住宅是未来的方向，请问公司是如何肯定的来确定？据媒体报道今年冬季的气温并没有直接显示气候...,0.038251
2,A000002,2010,"请问：王石董事长若离开万科,那万科会受到哪些影响呢?您好。王石主席并没有离开万科，在哈佛求学...",0.047258
3,A000002,2011,请问：在年报中提到２０１２年的结算收入的毛利率会有所下降，净利润会影响几个百分点．与２０１１...,0.034256
4,A000002,2012,万科的房子都是委托给谁来盖？公司没有下属的建筑公司，公司开发的物业都是外包给建筑公司承建，监...,0.039853


下图是论文中的Forward描述性统计，

![](img/stats.png)

我们试着看看分析结果 **df2['Forward']** 的

In [117]:
print('Forward最小值: ', df2['Forward'].min())
print('Forward中位数: ', df2['Forward'].median())
print('Forward均值: ', df2['Forward'].mean())
print('Forward最大值: ', df2['Forward'].max())
print('Forward标准层: ', df2['Forward'].std())

Forward最小值:  0.0
Forward中位数:  0.03411823223302536
Forward均值:  0.03474838140554545
Forward最大值:  0.15625
Forward标准层:  0.01179974855759022


可以发现描述性统计信息与论文的存在较大差异，可能的原因包括但不限于

1. 数据集存在差异。**论文中选取2007-2020年中小板和创业板上市公司作为研究对象。**而本实验使用的A股2005年-2023年所有的上市公司作为实验对象。
2. 可能筛选记录，文本太短的会议剔除。
3. 使用的停用词表不同
4. jieba导入自定义词典

